In [1]:
#!/usr/bin/python3
#
# This program and the accompanying materials
# are made available under the terms of the Apache License, Version 2.0
# which accompanies this distribution, and is available at
#
# http://www.apache.org/licenses/LICENSE-2.0

"""
This jupyter notebook is used to display the intersections
between various circles.

Developed for Covid-19 lockdown N°3 for the 10 km limitation

Requires the following packages:
- json: to open/read json files
- ipyleaflet: to display on map 
- ipywidgets: for map configuration
- shapely: for polygon operations
- geog: to compute polygon from center and radius
- numpy: required for geog

Input file: points.geojson
- GeoJSON file to store a list of points (centre of the each zone)

Ouput file: zone.geojson
- GeoJSON file to store the interesction zone as a Polygon

Hard-coded parameters:
- Radius: 10000m
- Number of points created for the circle: 32 points
- opacity values to display on map

"""
import json
from ipyleaflet import  GeoJSON, Map
from ipywidgets import Layout
from shapely.geometry import Polygon
import numpy as np
import geog
import shapely

def create_circle(lat, lon, radius, nb_points):
    """
    Create a circle from a point in WGS84 coordinate with 
    lat, lon: coordinates for the center
    radius: radius in m 
    nb_points: number of points for the circle
    """
    center = shapely.geometry.Point([lon, lat])
    angles = np.linspace(0, 360, nb_points)
    polygon = geog.propagate(center, angles, radius)
    return polygon

def display_circle(the_map, the_circle, name, options):
    """
    Display a circle on the map
    the_map: Ipleaflet Map
    the_circle: circle as a shapely Polygon
    name: name associated with the circle
    options: options to display circle
    """
    geo_circle = {
        "type": "Feature",
        "properties": {"name": name},
        "geometry": shapely.geometry.mapping(shapely.geometry.Polygon(the_circle))}
    layer = GeoJSON(data=geo_circle, style={'opacity': options["opacity"],
                                            'fillOpacity': options["fill_opacity"],
                                            'weight': options["weight"]})
    the_map.add_layer(layer)

def create_polygons(centers):
    """
    Create a list of shapely Polygon
    centers: list of points in a GeoJSON structure
    """
    polygon_circles = []
    for center in centers["features"]:
        lat = center["geometry"]["coordinates"][1]
        lon = center["geometry"]["coordinates"][0]
        polygon_circle = create_circle(lat, lon, 10000, 32)
        polygon_circles.append(Polygon(polygon_circle))
    return polygon_circles

def create_common_polygon(polygon_circles):
    """
    Create a Poygon
    polygon_circles: list of shapely Polygon
    """
    common_zone = polygon_circles[0]
    for circle in polygon_circles[1:]:
        common_zone = circle.intersection(common_zone)
    return common_zone

def generate_geojson_file(polygon, precision):
    """
    Generate a GeoJSON file fro a shapely Polygon
    polygon: shapely Polygon
    precision: number of digits for coordinates precision
    """
    geometry = shapely.geometry.mapping(shapely.geometry.Polygon(polygon))
    float_format = "{0:." + str(precision) + "f}"
    points = []
    for point in geometry["coordinates"][0]:
        lon = float(float_format.format(float(point[0])))
        lat = float(float_format.format(float(point[1])))
        points.append([lon, lat])
    polygon_coords = []
    polygon_coords.append(points)

    geo = {
        "type": "FeatureCollection",
        "properties": {"name": "Zone commune"},
        "features": [{
            "type": "Feature",
            "properties": {"name": "Cercle commun"},
            "geometry": {"type": "Polygon",
                         "coordinates": polygon_coords}}]
        }
    with open("zone.geojson", "w") as geojson_file:
        geojson_file.write(json.dumps(geo))

In [2]:
geo_centers = []
with open("points.geojson", "r") as geo_file:
   geo_centers = json.load(geo_file)
        
polygon_circles = create_polygons(geo_centers)
common_polygon = create_common_polygon(polygon_circles)
centroid = common_polygon.centroid

generate_geojson_file(common_polygon, 5)

# Create map centered on centroid
my_map = Map(center=(centroid.coords[0][1], centroid.coords[0][0]), 
             zoom=11, 
             layout=Layout(width='1200px', height='800px'))

# Display circles on the map
for circle in polygon_circles:
    display_circle(my_map, circle, "", {"opacity": 0.1, "fill_opacity": 0.1, "weight": 2})

# Display common zone on the map
display_circle(my_map, common_polygon, "Zone commune",
               {"opacity": 1.0, "fill_opacity": 0.5, "weight": 5})

# Display centers on the map
my_map.add_layer(GeoJSON(data=geo_centers))

# Display map
my_map

Map(center=[48.76338056406592, -3.466418124011323], controls=(ZoomControl(options=['position', 'zoom_in_text',…